###### 2020-11-22 일요일
###### 2020-11-23 월요일

In [127]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 01_텍스트데이터토큰화 & 토큰사전저장
 - 음절단위 토큰화 진행 과정에서 많은 시간이 소요된다
 - 그래서 의미없는 시간 손실을 줄이기 위해 음절단위 토큰화를 진행하고 음절단위 토큰사전을 json파일로 저장하고자 한다.

In [128]:
import warnings
warnings.filterwarnings(action='ignore')

import pandas as pd
import numpy as np
import re
import json
import time

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

## 1. 데이터 불러오기

In [129]:
comment_df = pd.read_csv('/content/drive/MyDrive/악성댓글_필터링_봇_모델/raw_data/comment_data.csv', encoding='UTF-8-SIG')
add_comment_df = pd.read_excel('/content/drive/MyDrive/악성댓글_필터링_봇_모델/raw_data/add_comment.csv')

comment_df = comment_df[ ['댓글', '악플여부'] ]

In [130]:
# 이 셀을 실행하면 추가한 데이터를 확인할 수 있음
# 데이터가 추가 되었는지 확인하고 진행하길 바람
add_comment_df.tail(20)

,댓글,악플여부
24935,말로만 죽는다 어쩐다 하더니 결국 안 죽고 살아있는 거 봐,2
24936,하 태생부터 근본없는것들은 개인방송 하지말자,2
24937,말로만 죽는다 어쩐다 하더니 결국 안 죽고 살아있는 거 봐,2
24938,하 태생부터 근본없는것들은 개인방송 하지말자,2
24939,말로만 죽는다 어쩐다 하더니 결국 안 죽고 살아있는 거 봐,2
24940,하 태생부터 근본없는것들은 개인방송 하지말자,2
24941,말로만 죽는다 어쩐다 하더니 결국 안 죽고 살아있는 거 봐,2
24942,하 태생부터 근본없는것들은 개인방송 하지말자\n말로만 죽는다 어쩐다 하더니 결국 안...,2
24943,하 태생부터 근본없는것들은 개인방송 하지말자,2
24944,말로만 죽는다 어쩐다 하더니 결국 안 죽고 살아있는 거 봐,2


In [131]:
# 하나의 데이터프레임으로 합치기
all_df = pd.concat([comment_df, add_comment_df])

# all_df = comment_df

In [132]:

# 라벨이 NaN인거 제거
index = all_df['악플여부'].isna()
raw_df = all_df[~index]

# 인덱스 리셋
raw_df = raw_df.reset_index()
raw_df.drop(['index'], inplace=True, axis=1)

In [133]:
# 원본유지 복사본 사용
copy_df = raw_df.copy()

In [134]:
# 고정 시드값 지정
seed = 123

# 댓글 길이 지정
comment_len = 400

#### 악플여부 라벨 1인 경우와 라벨 0인 경우를 하나로 합친다

In [135]:
index = copy_df['악플여부'] == 1
copy_df.loc[index, '악플여부'] = 0

index = copy_df['악플여부'] == 2
copy_df.loc[index, '악플여부'] = 1

In [136]:
copy_df['악플여부'] = copy_df['악플여부'].apply(int)
copy_df['댓글'] = copy_df['댓글'].apply(str)

In [137]:
copy_df['악플여부'].value_counts()

0    76691
1    38128
Name: 악플여부, dtype: int64

## 2. 토큰화 & 저장
 - Train set, Test set을 나눈다
 - Train set을 기준으로 음절 토큰 사전을 만든다
 - Train set / Test set / 음절토큰사전을 저장한다
    - 이렇게 저장하는 이유는? : 매번 음절토큰사전을 만드는데 시간이 많이 소요되기 때문에 한번 만들어놓고 재사용하여 시간을 절약!


In [138]:
def tokenization_save(df):

    feature = df['댓글']
    label = df['악플여부']

    X_train, X_test, y_train, y_test = train_test_split(feature, label, 
                                                    test_size=0.2,
                                                    random_state=seed)
    
    X_train_token_list = list(''.join(X_train))
    
    tokenizer = Tokenizer(oov_token = 'OOV')
    tokenizer.fit_on_texts(X_train_token_list)
    
    word_index_vocab = tokenizer.word_index

    return X_train, X_test, y_train, y_test, word_index_vocab


In [139]:
def train_test_save(X_train, X_test, y_train, y_test) :
    save_df_list = [X_train, X_test, y_train, y_test]
    file_names = ['X_train', 'X_test', 'y_train', 'y_test']
    for idx, set in enumerate(save_df_list):
        set.to_csv('/content/drive/MyDrive/악성댓글_필터링_봇_모델/04_최종모델 [1D-CNN]/data/' + file_names[idx] + '.csv', encoding='UTF-8-SIG')


def save_word_index(word_index_vocab) :
    json_file = json.dumps(word_index_vocab, ensure_ascii=False)
    f = open('/content/drive/MyDrive/악성댓글_필터링_봇_모델/04_최종모델 [1D-CNN]/data/word_index_vocab.json', 'w')
    f.write(json_file)
    f.close()

In [140]:
# 토큰화
start = time.time()

X_train, X_test, y_train, y_test, word_index_vocab = tokenization_save(copy_df)

print("걸린시간 :", time.time() - start)

걸린시간 : 15.778969526290894


In [141]:
# 공백도 음절로 포함
word_index_vocab[' '] = len(word_index_vocab)

In [142]:
# Train / Test 저장
train_test_save(X_train, X_test, y_train, y_test)

# 음절단위토큰 저장
save_word_index(word_index_vocab)